# TEAM 25 


## TASK 1: Name and Date Matching

This notebook goes thorugh the process of extracting bad actors from the given files using the names found on the open sactions website. 
A total of 17 names were finalized in this task as they were confirmed with name as well as date match. 

The names that matched without date of birth were separated out in a watchout list (unconfirmed matches)

### Setup

In [1]:
#!pip install Unidecode
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
from collections import Counter
import re
from unidecode import unidecode
from sklearn.feature_extraction.text import CountVectorizer
import torch
import datetime

### Function to calculate similarity scores

In [2]:
def sim_matrix(a, b, eps=1e-8):
    """
    added eps for numerical stability
    """
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n, device='cuda'))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n, device='cuda'))
    sim_mt = torch.mm(a_norm.to(device='cuda'), b_norm.to(device='cuda').transpose(0, 1))
    return sim_mt

### Read Data
1. Webscraped Data
2. UofT_nodes 

In [3]:
base_path = "C:/Users/dhava/University of Toronto/Tushar Verma - Big_data_2023/Dataset/"
df1 = pd.read_csv(base_path+'crime_data.csv')

df2 = pd.read_csv(base_path+'crime_fin_scrap_data.csv')

df3 = pd.read_csv(base_path+'crime_fraud_scrap_data.csv')

df4 = pd.read_csv(base_path+'crime_war_data.csv')

df5 = pd.read_csv(base_path+'terror_data.csv')

df_nodes = pd.read_csv(base_path+'UofT_nodes.csv').reset_index()

### Data Cleaning

In [4]:
split_symbol = ' . '
temp1 = df1['Birth date'].dropna().str.split(split_symbol).reset_index().explode('Birth date')
temp1 = temp1[~temp1['Birth date'].str.contains('more')]

temp2 = df1['Name'].str.split(split_symbol).reset_index().explode('Name').dropna()

df11 = temp1.merge(temp2, how='outer').dropna()


In [5]:
split_symbol = ' . '
temp1 = df1['Birth date'].dropna().str.split(split_symbol).reset_index().explode('Birth date')
temp1 = temp1[~temp1['Birth date'].str.contains('more')]

temp2 = df1['Other name'].str.split(split_symbol).reset_index().explode('Other name').dropna()

df12 = temp1.merge(temp2, how='outer').dropna()
df12.rename({'Other name':'Name'}, axis=1, inplace=True)

In [6]:
split_symbol = ' . '
temp1 = df1['Birth date'].dropna().str.split(split_symbol).reset_index().explode('Birth date')
temp1 = temp1[~temp1['Birth date'].str.contains('more')]

temp2 = df1['Weak alias'].str.split(split_symbol).reset_index().explode('Weak alias').dropna()

df13 = temp1.merge(temp2, how='outer').dropna()
df13.rename({'Weak alias':'Name'}, axis=1, inplace=True)

In [7]:
split_symbol = ' . '
temp1 = df2['Birth date'].dropna().str.split(split_symbol).reset_index().explode('Birth date')
#temp1 = temp1[~temp1['Birth date'].str.contains('more')]

temp2 = df2['Name'].str.split(split_symbol).reset_index().explode('Name').dropna()

df21 = temp1.merge(temp2, how='outer').dropna()


In [8]:
split_symbol = ' . '
df3['Birth date'] = 'not available'
temp1 = df3['Birth date'].dropna().str.split(split_symbol).reset_index().explode('Birth date')
#temp1 = temp1[~temp1['Birth date'].str.contains('more')]

temp2 = df3['Name'].str.split(split_symbol).reset_index().explode('Name').dropna()

df31 = temp1.merge(temp2, how='outer').dropna()


In [9]:
split_symbol = ' . '
temp1 = df4['Birth date'].dropna().str.split(split_symbol).reset_index().explode('Birth date')
temp1 = temp1[~temp1['Birth date'].str.contains('more')]

temp2 = df4['Name'].str.split(split_symbol).reset_index().explode('Name').dropna()

df41 = temp1.merge(temp2, how='outer').dropna()


In [10]:
split_symbol = ' . '
temp1 = df4['Birth date'].dropna().str.split(split_symbol).reset_index().explode('Birth date')
temp1 = temp1[~temp1['Birth date'].str.contains('more')]

temp2 = df4['Other name'].str.split(split_symbol).reset_index().explode('Other name').dropna()

df42 = temp1.merge(temp2, how='outer').dropna()
df42.rename({'Other name':'Name'}, axis=1, inplace=True)

In [11]:
split_symbol = ' . '
temp1 = df4['Birth date'].dropna().str.split(split_symbol).reset_index().explode('Birth date')
temp1 = temp1[~temp1['Birth date'].str.contains('more')]

temp2 = df4['Weak alias'].str.split(split_symbol).reset_index().explode('Weak alias').dropna()

df43 = temp1.merge(temp2, how='outer').dropna()
df43.rename({'Weak alias':'Name'}, axis=1, inplace=True)

In [12]:
split_symbol = ' . '
temp1 = df5['Birth date'].dropna().str.split(split_symbol).reset_index().explode('Birth date')
temp1 = temp1[~temp1['Birth date'].str.contains('more')]

temp2 = df5['Name'].str.split(split_symbol).reset_index().explode('Name').dropna()

df51 = temp1.merge(temp2, how='outer').dropna()


In [13]:
split_symbol = ' . '
temp1 = df5['Birth date'].dropna().str.split(split_symbol).reset_index().explode('Birth date')
temp1 = temp1[~temp1['Birth date'].str.contains('more')]

temp2 = df5['Other name'].str.split(split_symbol).reset_index().explode('Other name').dropna()

df52 = temp1.merge(temp2, how='outer').dropna()
df52.rename({'Other name':'Name'}, axis=1, inplace=True)

In [14]:
split_symbol = ' . '
temp1 = df5['Birth date'].dropna().str.split(split_symbol).reset_index().explode('Birth date')
temp1 = temp1[~temp1['Birth date'].str.contains('more')]

temp2 = df5['Weak alias'].str.split(split_symbol).reset_index().explode('Weak alias').dropna()

df53 = temp1.merge(temp2, how='outer').dropna()
df53.rename({'Weak alias':'Name'}, axis=1, inplace=True)

In [15]:
df_nodes_names = df_nodes[['index', 'NAME', 'RISK', 'BIRTH_DT']]

df_nodes_names = df_nodes_names.dropna(subset=['NAME'])
df_nodes_names['NAME_PROCESSED'] = df_nodes_names['NAME'].str.lower().apply(lambda x:unidecode(x))
df_nodes_names['NAME_PROCESSED'] = df_nodes_names['NAME_PROCESSED'].apply(lambda x:re.sub(r'[^a-zA-Z ]+', '', x))

df_nodes_names['NAME_PROCESSED_SP'] = df_nodes_names['NAME_PROCESSED'].str.replace(' ', '')
df_nodes_names = df_nodes_names.dropna().reset_index(drop=True) 
lens = df_nodes_names['NAME_PROCESSED'].apply(lambda x:len(x))
df_nodes_names = df_nodes_names[lens>0]
df_nodes_names = df_nodes_names.dropna().reset_index(drop=True)

In [16]:
df_nodes_names.to_csv('all_names.csv')

### Combining Cleaned Data

In [17]:
df = pd.concat([df11[['Name', 'Birth date']], 
                df12[['Name', 'Birth date']], 
                df13[['Name', 'Birth date']], 
                df21[['Name', 'Birth date']],
                df31[['Name', 'Birth date']],
                df41[['Name', 'Birth date']],
                df42[['Name', 'Birth date']],
                df43[['Name', 'Birth date']],
                df51[['Name', 'Birth date']],
                df52[['Name', 'Birth date']],
                df53[['Name', 'Birth date']]], axis=0).drop_duplicates().dropna().reset_index(drop=True)


In [18]:
df_pr = df.dropna(subset=['Name'])
df_pr['NAME_PROCESSED'] = df_pr['Name'].str.lower().apply(lambda x:unidecode(x))
df_pr['NAME_PROCESSED'] = df_pr['NAME_PROCESSED'].apply(lambda x:re.sub(r'[^a-zA-Z ]+', '', x))

df_pr = df_pr.dropna().reset_index(drop=True) 
df_pr

,Name,Birth date,NAME_PROCESSED
0,Mr Stewart Andrew Inglis,1962-02-27,mr stewart andrew inglis
1,Stewart Andrew INGLIS,1962-02-27,stewart andrew inglis
2,"BARKHANOEV, Malik Ruslanovish",1992-03-14,barkhanoev malik ruslanovish
3,MALIK BARKHANOEV,1992-03-14,malik barkhanoev
4,MALIK RUSLANOVICH BARKHANOEV,1992-03-14,malik ruslanovich barkhanoev
...,...,...,...
26386,Maulavi Adam),1974,maulavi adam
26387,Maulawi ADAM,1974,maulawi adam
26388,Molawi ADAM,1974,molawi adam
26389,マウラヴィ・アダム,1974,mauravuiadamu


### Extracting 'Also Known As' Data

In [19]:
extra_names = [' aka ', 'aka ', ' aka', 'known', 'also known as', 'formerly known as']
df_pr[df_pr['NAME_PROCESSED'].apply(lambda x:sum([(char in x.lower()) for char in extra_names])) !=0]


,Name,Birth date,NAME_PROCESSED
68,Rajinder Singh BHATIA (AKA RAJVEER SINGH TAKAWAL),not available,rajinder singh bhatia aka rajveer singh takawal
132,Jamie HUMPHRIES (AKA ANDERSON),1985-12-10,jamie humphries aka anderson
136,Nishar Ahmed ISMAIL (AKA BATHA),1967-04-15,nishar ahmed ismail aka batha
137,Nishar Ahmed Ismail BATHA (AKA ISMAIL),1967-04-15,nishar ahmed ismail batha aka ismail
413,Hannah BRADLEY AKA HANNAH SUSAN CLARK,1985-03-15,hannah bradley aka hannah susan clark
...,...,...,...
25695,"（a.k.a. Salem Ali, Fahd Bin Adballah Bin Khali...",1964-03-01,aka salem ali fahd bin adballah bin khalid ash...
25697,"（a.k.a. Salem Ali, Fahd Bin Adballah Bin Khali...",1964-04-14,aka salem ali fahd bin adballah bin khalid ash...
25699,"（a.k.a. Salem Ali, Fahd Bin Adballah Bin Khali...",1965-04-14,aka salem ali fahd bin adballah bin khalid ash...
26192,（a.k.a.: Jamal Lounici）,01-02-1962,aka jamal lounici


In [20]:
import re

def split_text(data):
    return re.split('also known as|formerly known as| aka|aka | aka  ', data)
    
    

In [21]:
df_pr['NEW'] = df_pr['NAME_PROCESSED'].apply(lambda x:split_text(x))
df_new = df_pr.explode('NEW')

In [22]:
df_new['NEW_SPACE'] = df_new['NEW'].str.replace(' ', '')

### Bringing Dates to a consistent format

In [86]:
def parser_y(s):
    
    if len(s)==10:
        check = s.split('-')[-1]
        if len(check)!=4:
            format_q='%Y-%m-%d' 
            return datetime.strptime(s, format_q).date()
        else:
            format_q='%d-%m-%Y' 
            return datetime.strptime(s, format_q).date()
        
    elif len(s)==4:
        format_q='%Y'
        return datetime.strptime(s, format_q).date()
    else:
        check = int(s.split('-')[-1])
        if check>12:
            format_q='%Y-%d'
            return datetime.strptime(s, format_q).date()
        else:
            format_q='%Y-%m'
            return datetime.strptime(s, format_q).date()    

In [89]:
from datetime import datetime
df_rem = df_new[df_new['NEW_SPACE']!='more']
df_rem = df_rem[df_rem['Birth date']!='not available']
df_rem['DOB'] = df_rem['Birth date'].apply(lambda x: parser_y(x))
df_rem['AGE'] = 2022 - df_rem['DOB'].apply(lambda x: x.year)
df_rem

,Name,Birth date,NAME_PROCESSED,NEW,NEW_SPACE,DOB,AGE
0,Mr Stewart Andrew Inglis,1962-02-27,mr stewart andrew inglis,mr stewart andrew inglis,mrstewartandrewinglis,1962-02-27,60
1,Stewart Andrew INGLIS,1962-02-27,stewart andrew inglis,stewart andrew inglis,stewartandrewinglis,1962-02-27,60
2,"BARKHANOEV, Malik Ruslanovish",1992-03-14,barkhanoev malik ruslanovish,barkhanoev malik ruslanovish,barkhanoevmalikruslanovish,1992-03-14,30
3,MALIK BARKHANOEV,1992-03-14,malik barkhanoev,malik barkhanoev,malikbarkhanoev,1992-03-14,30
4,MALIK RUSLANOVICH BARKHANOEV,1992-03-14,malik ruslanovich barkhanoev,malik ruslanovich barkhanoev,malikruslanovichbarkhanoev,1992-03-14,30
...,...,...,...,...,...,...,...
26385,Maulavi Adam Khan,1974,maulavi adam khan,maulavi adam khan,maulaviadamkhan,1974-01-01,48
26386,Maulavi Adam),1974,maulavi adam,maulavi adam,maulaviadam,1974-01-01,48
26387,Maulawi ADAM,1974,maulawi adam,maulawi adam,maulawiadam,1974-01-01,48
26388,Molawi ADAM,1974,molawi adam,molawi adam,molawiadam,1974-01-01,48


In [94]:
df_nodes_names['DOB'] = df_nodes_names['BIRTH_DT'].apply(lambda x: parser_y(x))
df_nodes_names['AGE'] = 2022 - df_nodes_names['DOB'].apply(lambda x: x.year)
df_nodes_names

,Unnamed: 0,index,NAME,RISK,BIRTH_DT,NAME_PROCESSED,NAME_PROCESSED_SP,DOB,AGE
0,0,0,"Young, Marie Mildren Coleman",low,1981-09-01,young marie mildren coleman,youngmariemildrencoleman,1981-09-01,41
1,1,1,Mark Stupar Lecy,low,1994-02-21,mark stupar lecy,markstuparlecy,1994-02-21,28
2,2,2,Dean Glasper Wendel Reeves,low,1962-11-16,dean glasper wendel reeves,deanglasperwendelreeves,1962-11-16,60
3,3,3,"Hulsey, Linda Rauth",low,1998-06-20,hulsey linda rauth,hulseylindarauth,1998-06-20,24
4,4,4,Carolyn Washington Roberts,low,1942-01-24,carolyn washington roberts,carolynwashingtonroberts,1942-01-24,80
...,...,...,...,...,...,...,...,...,...
999333,999333,999995,Bret* Dove Gainey,high,1939-01-16,bret dove gainey,bretdovegainey,1939-01-16,83
999334,999334,999996,Anthony Ray Montano Gomez,low,1970-12-13,anthony ray montano gomez,anthonyraymontanogomez,1970-12-13,52
999335,999335,999997,Bernadette Schofield Pace Mile,high,1958-09-09,bernadette schofield pace mile,bernadetteschofieldpacemile,1958-09-09,64
999336,999336,999998,"Wasson, Russell Terrell",high,1983-09-15,wasson russell terrell,wassonrussellterrell,1983-09-15,39


### Name and Date matching

In [92]:
df_rem.to_csv('with_age.csv') #includes age column for bad actors, dropped not available dates

In [23]:
df_new.to_csv('bad_actors.csv') #without age column

In [24]:
df_rem = pd.read_csv('with_age.csv')
df_nodes_names = pd.read_csv('all_names.csv')
all_names = df_nodes_names['NAME_SPACE']

In [134]:
df_rem = df_rem.reset_index()

In [135]:
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 2))
bad_actors_vecs = vectorizer.fit_transform(df_rem['NEW_SPACE'])
all_names_vecs = vectorizer.transform(df_nodes_names['NAME_PROCESSED_SP'])

### Storing matched names in respestive dataframes

In [203]:
internet = []
uoft = []
a = torch.tensor(bad_actors_vecs.toarray(), device='cuda', dtype=torch.float16)
step = 5000
matching_df = pd.DataFrame()
for i in tqdm.tqdm(range(0, 1000000, step)):
    b = torch.tensor(all_names_vecs[i:i+step].toarray(), device='cuda', dtype=torch.float16)
    res = sim_matrix(a, b)
    index = torch.where(res>0.9)
    internet.append(list(index[0].cpu().numpy()))
    uoft.append(list(index[1].cpu().numpy()+i))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:13<00:00, 15.03it/s]


In [160]:
import itertools
internet_o = list(itertools.chain(*internet))
uoft_o = list(itertools.chain(*uoft))

In [177]:
df_rem.iloc[internet_o].add_prefix('inter_')

,inter_index,inter_Name,inter_Birth date,inter_NAME_PROCESSED,inter_NEW,inter_NEW_SPACE,inter_DOB,inter_AGE
8929,8998,EVAN NEUMANN,1972-06-22,evan neumann,evan neumann,evanneumann,1972-06-22,50
7961,8012,DANIEL ANDREAS SAN DIEGO,1978-02-09,daniel andreas san diego,daniel andreas san diego,danielandreassandiego,1978-02-09,44
7962,8013,Daniel Andreas San Diego,1978-02-09,daniel andreas san diego,daniel andreas san diego,danielandreassandiego,1978-02-09,44
9157,9237,WILLIAM WILLINGHAM,1961-06-21,william willingham,william willingham,williamwillingham,1961-06-21,61
786,818,Amanda Jane EWAN (AKA STANLEY),1973-07-24,amanda jane ewan aka stanley,stanley,stanley,1973-07-24,49
...,...,...,...,...,...,...,...,...
16025,23897,"'ASIRI, Ibrahim Hasan Tali'A",1982-04-19,asiri ibrahim hasan talia,asiri ibrahim hasan talia,asiriibrahimhasantalia,1982-04-19,40
16028,23900,"AL ASIRI, Ibrahim Hassan",1982-04-19,al asiri ibrahim hassan,al asiri ibrahim hassan,alasiriibrahimhassan,1982-04-19,40
7954,8005,JONG GIL,1962-08-30,jong gil,jong gil,jonggil,1962-08-30,60
7956,8007,Jong Gil,1962-08-30,jong gil,jong gil,jonggil,1962-08-30,60


In [176]:
df_nodes_names.iloc[uoft_o].add_prefix('uoft_')

,uoft_Unnamed: 0,uoft_index,uoft_NAME,uoft_RISK,uoft_BIRTH_DT,uoft_NAME_PROCESSED,uoft_NAME_PROCESSED_SP,uoft_DOB,uoft_AGE
2169,2169,2173,Anne Evan,low,1980-09-06,anne evan,anneevan,1980-09-06,42
9650,9650,9659,Daniel Andreas,low,1918-12-05,daniel andreas,danielandreas,1918-12-05,104
9650,9650,9659,Daniel Andreas,low,1918-12-05,daniel andreas,danielandreas,1918-12-05,104
7152,7152,7160,"Williams, William Mclean Ling",low,1950-10-22,williams william mclean ling,williamswilliammcleanling,1950-10-22,72
14562,14562,14576,"Martin, Stanley Stanley",low,1973-12-27,martin stanley stanley,martinstanleystanley,1973-12-27,49
...,...,...,...,...,...,...,...,...,...
993602,993602,994261,"Assiri, Ibrahim Hassan Tali",high,1982-04-19,assiri ibrahim hassan tali,assiriibrahimhassantali,1982-04-19,40
993602,993602,994261,"Assiri, Ibrahim Hassan Tali",high,1982-04-19,assiri ibrahim hassan tali,assiriibrahimhassantali,1982-04-19,40
996315,996315,996975,O Jong Gil,high,1962-08-30,o jong gil,ojonggil,1962-08-30,60
996315,996315,996975,O Jong Gil,high,1962-08-30,o jong gil,ojonggil,1962-08-30,60


### Saving the matched names for further analysis

In [182]:
df_tushar = pd.concat([df_rem.iloc[internet_o,[1, 5,6,7]].add_prefix('inter_').reset_index(),df_nodes_names.iloc[uoft_o,[0, 1,3, 6,7,8]].add_prefix('uoft_').reset_index()], axis=1)
df_tushar.to_csv('refer_bad_actors.csv')